<a href="https://colab.research.google.com/github/Shady91-ML/Fake-news-ML/blob/main/Fake_news_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading libraries


In [87]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [88]:
#Download all stopwords which are not significant and can be removed from data
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data pre-processing


In [89]:
news_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project 4 - Fake news ML/fake_train.csv')
all_news_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project 4 - Fake news ML/fake_train.csv')

# Data analysis

In [90]:
# Checking first rows of data:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [91]:
#Checking number of rows and columns:
news_df.shape

(20800, 5)

In [92]:
#Checking missing values:
news_df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [93]:
#Replacing missing values with empty string:
news_df = news_df.fillna(' ')

In [94]:
#Merging title and author
news_df['content']=news_df['author']+' '+news_df['title']

# Data split


Stemming - reducing a word to its root word

In [95]:
#Initializing PorterStremmer
port_stemmer = PorterStemmer()
#Creating function
def stemming(content):
  #Removing all numbers, symbols:
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  #Making everything small:
  stemmed_content = stemmed_content.lower()
  #Converting to list
  stemmed_content = stemmed_content.split()
  #Stemming, reducing to root word and removing stopwords
  stemmed_content = [port_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #Joining:
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [96]:
news_df['content'] = news_df['content'].apply(stemming)

In [97]:
news_df['content'].head()

,content
0,darrel lucu hous dem aid even see comey letter...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...


In [98]:
#Separeting content and label
X = news_df['content'].values
y = news_df['label'].values

In [99]:
#Converting text to numerical data:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1968)

# ML model training


In [101]:
#Initializing model:
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)

LogisticRegression()

# Model evaluation

In [102]:
#Accuracy score on training data:
predict_train = logistic_regression_model.predict(X_train)
score_train = accuracy_score(predict_train, y_train)

print('Accuracy score on training data: ', score_train)

Accuracy score on training data:  0.9870793269230769


In [103]:
#Accuracy score on test data:
predict_test = logistic_regression_model.predict(X_test)
score_test = accuracy_score(predict_test, y_test)

print('Accuracy score on training data: ', score_test)

Accuracy score on training data:  0.9757211538461539


#Predictive system


In [104]:
X_news = (all_news_df.iloc[15])
X_news['content']=X_news['author']+' '+ X_news['title']
X_news['content'] =stemming(X_news['content'])
X_new_vecc = pd.Series(X_news['content'])
vec_X = vectorizer.transform(X_new_vecc)
news = logistic_regression_model.predict(vec_X)
if news == 1:
  print('It is fake news')
else:
  print('It is real news')

It is real news


<ipython-input-104-9d6f490c2b24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_news['content']=X_news['author']+' '+ X_news['title']
<ipython-input-104-9d6f490c2b24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_news['content']=X_news['author']+' '+ X_news['title']
<ipython-input-104-9d6f490c2b24>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_news['content'] =stemming(X_news['content'])
